# Cleaning csvs

In [1]:
from os import path
import os

import pandas as pd

In [2]:
os.makedirs('../data/output/', exist_ok=True)

## Broward

In [3]:
broward = pd.read_csv('../data/input/foia/broward.csv')
characteristics = pd.read_csv('../data/intermediary/scorecard/characteristics/broward.csv')
equity = pd.read_csv('../data/intermediary/scorecard/equity/broward.csv')
keys = pd.read_csv('../data/intermediary/keys/broward.csv')

In [4]:
broward = broward.rename(columns={'SchoolName': 'school_name'})
broward = broward.merge(keys[['school_name', 'school_number']], on='school_name')

In [5]:
historical = pd.read_csv('../data/input/historical_attendance_1819.csv')
historical = historical[historical['DISTRICT #'] == keys.district_number[0]]
historical.columns

hist_columns = [
    'DAYS PRESENT', 
    'DAYS ABSENT', 
    'TOTAL DAYS', 
    '% PRESENT', 
    'AVERAGE DAILY ATTENDANCE', 
    'AVERAGE DAILY MEMBERSHIP'
]

for column in hist_columns:
    historical[column] = pd.to_numeric(historical[column].str.replace(',','').str.replace('%',''))
    
hist_columns.append("SCHOOL #")

In [6]:
historical_context = pd.merge(historical, equity, left_on="SCHOOL #", right_on="school_number")

historical_poverty = historical_context.groupby('poverty').sum()
historical_poverty['% PRESENT'] = historical_poverty['DAYS PRESENT'] / historical_poverty['TOTAL DAYS']
historical_poverty

,DISTRICT #,SCHOOL #,DAYS PRESENT,DAYS ABSENT,TOTAL DAYS,% PRESENT,AVERAGE DAILY ATTENDANCE,AVERAGE DAILY MEMBERSHIP,school_number,district_number
poverty,,,,,,,,,,
High Poverty,348,132110,5242658,396615,5639273,0.929669,29126,31325,132110,348
Low Poverty,348,232756,10748908,533463,11282371,0.952717,59714,62680,232756,348
Mid-Range Poverty,1170,531094,29389570,2097240,31486810,0.933393,163280,174929,531094,1170


In [7]:
export = broward.groupby(['school_name', 'school_number']).sum()
export = export.reset_index()
export['covid.agg.enroll'] = export['Enrollment']
export['covid.agg.attend'] = export['Login']
export = export[['school_name', 'school_number', 'covid.agg.enroll', 'covid.agg.attend']]
export['covid.avg'] = export['covid.agg.attend'] / export['covid.agg.enroll'] * 100

In [8]:
historical = historical.rename(columns={
    "SCHOOL #":      "school_number",
    "DISTRICT #":    "district_number",
    "DISTRICT NAME": "district_name",
    "DAYS PRESENT":  "1819.agg.attend",
    "TOTAL DAYS":    "1819.agg.enroll"
})

historical['1819.avg'] = historical["1819.agg.attend"] / historical["1819.agg.enroll"] * 100

export_hist = historical[[
    "school_number",
    "district_number",
    "district_name",
    "1819.agg.attend",
    "1819.agg.enroll",
    "1819.avg"
]]

export = export.merge(export_hist, on='school_number')

In [9]:
export = export.merge(equity, on=['school_number', 'district_number'])
export = export.merge(characteristics[[
    'school_number',
    'school_type', 
    'grade'
]], on='school_number')

In [10]:
export

,school_name,school_number,covid.agg.enroll,covid.agg.attend,covid.avg,district_number,district_name,1819.agg.attend,1819.agg.enroll,1819.avg,minority,poverty,title_i,school_type,grade
0,ANNABEL C. PERRY PK-8,1631,7381,6667,90.326514,6,BROWARD,124508,131620,94.596566,High Minority,High Poverty,Title I,Combination School,B
1,APOLLO MIDDLE,1791,15320,14050,91.710183,6,BROWARD,237032,250084,94.780954,Mid-Range Minority,Mid-Range Poverty,Title I,Middle School,B
2,ATLANTIC TECHNICAL HIGH,2221,6622,6361,96.058593,6,BROWARD,119041,123474,96.409770,Mid-Range Minority,Mid-Range Poverty,Non-Title I,High School,A
3,ATLANTIC WEST ELEMENTARY,2511,6748,6071,89.967398,6,BROWARD,120886,128444,94.115724,High Minority,Mid-Range Poverty,Title I,Elementary School,C or below
4,ATTUCKS MIDDLE,343,9040,8165,90.320796,6,BROWARD,136578,143403,95.240685,Mid-Range Minority,Mid-Range Poverty,Title I,Middle School,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,WHISPERING PINES EXCEPTIONAL EDUCATION CENTER,1752,1483,987,66.554282,6,BROWARD,35191,39577,88.917806,Mid-Range Minority,Mid-Range Poverty,Non-Title I,Combination School,Improvement Plan
214,WILLIAM DANDY MIDDLE,1071,9801,7960,81.216202,6,BROWARD,170472,173839,98.063150,High Minority,High Poverty,Title I,Middle School,C or below
215,WILLIAM T. MCFATTER TECHNICAL HIGH,1291,6688,6204,92.763158,6,BROWARD,106202,111249,95.463330,Mid-Range Minority,Mid-Range Poverty,Non-Title I,High School,A
216,WILTON MANORS ELEMENTARY,191,6303,5773,91.591306,6,BROWARD,103377,108679,95.121413,Mid-Range Minority,Mid-Range Poverty,Title I,Elementary School,A


In [11]:
export.to_csv("../data/output/broward.csv", index=False)

## Miami-Dade

In [12]:
miami = pd.read_csv('../data/input/foia/miami-dade.csv')
characteristics = pd.read_csv('../data/intermediary/scorecard/characteristics/miami-dade.csv')
equity = pd.read_csv('../data/intermediary/scorecard/equity/miami-dade.csv')
enrollment = pd.read_csv('../data/intermediary/scorecard/enrollment/miami-dade.csv')

In [13]:
df = miami.merge(enrollment, on='school_number')
df['total_students'] = pd.to_numeric(df['total_students'])

df['est_present'] = df['pct_present'] / 100.0 * df['total_students']

In [14]:
historical = pd.read_csv('../data/input/historical_attendance_1819.csv')
historical = historical[historical['DISTRICT NAME'] == 'MIAMI-DADE']
historical.columns

hist_columns = [
    'DAYS PRESENT',
    'TOTAL DAYS', 
    '% PRESENT',
]

for column in hist_columns:
    historical[column] = pd.to_numeric(historical[column].str.replace(',','').str.replace('%',''))
    
hist_columns.append("SCHOOL #")

In [15]:
historical = historical[hist_columns]
historical = historical.rename(columns={
    'TOTAL DAYS':'1819.agg.enroll',
    'DAYS PRESENT':'1819.agg.attend',
    '% PRESENT': '1819.avg',
    'SCHOOL #':'school_number'
})

In [16]:
export = df.groupby(['school_name', 'school_number']).sum()
export = export.reset_index()

In [17]:
export = export[['school_name', 'school_number', 'total_students', 'est_present']]
export = export.merge(characteristics, on="school_number")
export = export.merge(equity, on=["school_number", 'district_number'])

export = export[[
    "school_name",
    "school_number",
    "total_students",
    "est_present",
    "school_type",
    "grade",
    "charter",
    "alt_schl",
    "minority",
    "poverty",
    "title_i"]
]

export = export.merge(historical, on='school_number')

export = export.rename(columns={
    'total_students': 'covid.agg.enroll',
    'est_present': 'covid.agg.attend'
})

export['covid.avg'] = export['covid.agg.attend'] / export['covid.agg.enroll'] * 100

In [18]:
export.to_csv("../data/output/miami-dade.csv", index=False)